In [1]:
from nltk import PCFG, ViterbiParser, InsideChartParser

In [6]:
grammer = PCFG.fromstring("""
S -> NP VP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
NP -> Det N [0.5] | NP PP [0.2] | 'John' [0.3]
PP -> P NP [1.0]
V -> 'saw' [0.2] | 'ate' [0.8]
Det -> 'the' [0.6] | 'a' [0.4]
N -> 'man' [0.5] | 'telescope' [0.5]
P -> 'with' [0.4] | 'in' [0.6]
""")

parser1 = ViterbiParser(grammer)
parser2 = InsideChartParser(grammer)

sentence = "John saw a man with a telescope"
tokens = sentence.split()

for tree in parser1.parse(tokens):
  print(tree)

for tree in parser2.parse(tokens):
  print(tree)

(S
  (NP John)
  (VP
    (VP (V saw) (NP (Det a) (N man)))
    (PP (P with) (NP (Det a) (N telescope))))) (p=5.04e-05)
(S
  (NP John)
  (VP
    (VP (V saw) (NP (Det a) (N man)))
    (PP (P with) (NP (Det a) (N telescope))))) (p=5.04e-05)
(S
  (NP John)
  (VP
    (V saw)
    (NP
      (NP (Det a) (N man))
      (PP (P with) (NP (Det a) (N telescope)))))) (p=3.36e-05)


In [7]:
import numpy as np

In [9]:
def cyk_parse(grammar, sentence):
    n= len(sentence)
    table =np.empty((n, n), dtype=object)
    for i, word in enumerate(sentence):
        table[i, i] = [(p.lhs(), p.prob()) for p in grammar.productions() if len(p.rhs()) == 1 and p.rhs()[0] == word]
    for span in range(2, n + 1):
        for i in range(n - span + 1):
            table[i, i + span - 1] = [(p.lhs(), lp * rp * p.prob()) for k in range(i, i + span - 1)
                for l, lp in (table[i, k] or []) for r, rp in (table[k + 1, i + span - 1] or [])
                for p in grammar.productions() if len(p.rhs()) == 2 and (l, r) == p.rhs()]
    return table
def check_sentence(sentence, grammar):
    words, results = sentence.split(), cyk_parse(grammar, sentence.split())[0, len(sentence.split()) - 1] or []
    prob = max((p for nt, p in results if str(nt) == 'S'), default=None)
    print(f"Grammatically correct. Max probability: {prob}" if prob else "Not grammatically correct.")
    return prob

grammar = PCFG.fromstring("""
S -> NP VP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
NP -> Det N [0.5] | NP PP [0.2] | 'John' [0.3]
PP -> P NP [1.0]
V -> 'saw' [0.2] | 'ate' [0.8]
Det -> 'the' [0.6] | 'a' [0.4]
N -> 'man' [0.5] | 'telescope' [0.5]
P -> 'with' [0.4] | 'in' [0.6]
""")
if grammar:
    final_prob = check_sentence('John saw a man with a telescope', grammar)


Grammatically correct. Max probability: 5.040000000000001e-05
